# Repository Design Pattern

Repository Depo manasına gelen bir design patterndir. Bir çok diğer pattern'de olduğu gibi ana amaç parçalar arası bağımlılığı azaltmaktır. 
    

Depo burada verilerin tutulduğu database manasındadır. Bu depoya sürekli her çalışanın girdiği bir firmada kaos kaçınılmazdır. Bu yüzden firmalarda depodan sorumlu ekip farklıdır. Yazılımda da database ile bağlantıyı her ihtiyaç duyulan noktadan ayrı ayrı kurarsak karmaşa kaçınılmazdır. Bunun yerine bir database erişim servisi yazarak erişimi tek noktadan yönetmek ve o programın içinde bu erişim servisi ile database ulaşımını sağlamak her şeyi daha kontrol edilebilir yapar.    

Bu örnekte bir database kurmak yerine bir python sözlüğüne database muamelesi yapacağız.  

In [ ]:
dataBase={
            1: {"name": "Alice", "age": 30, "email": "alice@example.com"},
            2: {"name": "Bob", "age": 25, "email": "bob@example.com"},
            3: {"name": "Charlie", "age": 35, "email": "charlie@example.com"},
        }

keyler id oldu ve biz id kullanarak iç bilgilere erişeceğiz, elbette gerçek bir database böyle değildir ama amacımız konsepti anlamak.

Aşağıda verilen repositorysiz koda bakalım

In [ ]:
# Basit bir data base objesi
class SimpleMockDatabase:
    def __init__(self):
        self.data = {
            1: {"name": "Alice", "age": 30, "email": "alice@example.com"},
            2: {"name": "Bob", "age": 25, "email": "bob@example.com"},
            3: {"name": "Charlie", "age": 35, "email": "charlie@example.com"},
        }
        self.next_id = 4

# data base bilgilerine erişim kodları var
    def get_all(self):
        return list(self.data.values())

    def get_by_id(self, record_id):
        return self.data.get(record_id, None)

    def add(self, record):
        record_id = self.next_id
        self.data[record_id] = record
        self.next_id += 1
        return record_id

    def update(self, record_id, updated_record):
        if record_id in self.data:
            self.data[record_id].update(updated_record)
            return True
        return False

    def delete(self, record_id):
        return self.data.pop(record_id, None) is not None

# uygulama içinde data base direkt olarak kullanılmış
# bunlar bir grup değil uygulamana yayılmış olarak bulunuyor 
# (eğer bir yere gruplarsan repository olacak zaten)
# farklı sayfalarda bu fonksiyonların kullanıldığını düşün
#  bir değişiklikte her yerde bu kodların olduğunu düşün

def list_users():
    db = SimpleMockDatabase()
    return db.get_all()

def find_user(user_id):
    db = SimpleMockDatabase()
    return db.get_by_id(user_id)

def create_user(name, age, email):
    db = SimpleMockDatabase()
    return db.add({"name": name, "age": age, "email": email})

def update_user(user_id, name=None, age=None, email=None):
    db = SimpleMockDatabase()
    return db.update(user_id, {"name": name, "age": age, "email": email})

def delete_user(user_id):
    db = SimpleMockDatabase()
    return db.delete(user_id)

# kod içinde bu fonkisyonların kullanımı 
print(list_users())
print(find_user(1))
print(create_user("Diana", 28, "diana@example.com"))
print(update_user(1, age=31))
print(delete_user(2))

yaşanabilecek bir problem


In [ ]:
def list_users():
    db = SimpleMockDatabase()
    return db.get_all()

fonksiyon direkt data base ile ilişkiye geçiyor ancak farklı bir database geçişi yapacak olsak bu fonksiyonları yeniden yazmamız gerekecek.

In [ ]:
def test_find_user():
    db = SimpleMockDatabase()
    db.add({"name": "Test User", "age": 30, "email": "test@example.com"})
    result = db.get_by_id(1)
    assert result["name"] == "Test User"


bir test oluşturmak için database içine direk erişiliyor ancak bunun için databaselere direkt ulaşma  ayrı bir connection oluşturmak gerekiyor. 

Tüm database bağlantılarını tek bir merkeze toplama : repository design pattern

In [ ]:
# Database erişimi tek bir class üzerinden yönetilir.
class UserRepository:
    def __init__(self):
        self.db = SimpleMockDatabase()

    def get_all(self):
        return self.db.get_all()

    def get_by_id(self, record_id):
        return self.db.get_by_id(record_id)

    def add(self, record):
        return self.db.add(record)

    def update(self, record_id, updated_record):
        return self.db.update(record_id, updated_record)

    def delete(self, record_id):
        return self.db.delete(record_id)

# Kodun içindeki kodlar direkt database ulaşımı yerine 
# senin oluşturduğun data base  erişim sınıfıyla çalışır 
def list_users(repo):
    return repo.get_all()

def find_user(repo, user_id):
    return repo.get_by_id(user_id)

def create_user(repo, name, age, email):
    return repo.add({"name": name, "age": age, "email": email})

def update_user(repo, user_id, name=None, age=None, email=None):
    return repo.update(user_id, {"name": name, "age": age, "email": email})

def delete_user(repo, user_id):
    return repo.delete(user_id)

# reponu oluşturduktan sonra bunu kullanırsın eğer 
# database değişirse reponu değiştir.
repo = UserRepository()
print(list_users(repo))
print(find_user(repo, 1))
print(create_user(repo, "Diana", 28, "diana@example.com"))
print(update_user(repo, 1, age=31))
print(delete_user(repo, 2))
